In [1]:
"""
Dataset Analysis Script untuk MIT-BIH AF Dataset
Menganalisis komposisi dataset dan memberikan rekomendasi untuk single-annotation records
"""

import os
import glob
import wfdb
import numpy as np
import pandas as pd
from collections import Counter

# Configuration - SESUAIKAN dengan path Anda
DATA_DIR = 'D:\\skripsi_teknis\\dataset\\mitbih-afdb'

def get_available_records():
    """
    Get list of available MIT-BIH AF records dari path lokal
    """
    print(f"Scanning for records in: {DATA_DIR}")
    
    # Cari file .dat untuk mendapatkan record IDs yang tersedia
    dat_files = glob.glob(os.path.join(DATA_DIR, "*.dat"))
    
    if not dat_files:
        print(f"❌ No .dat files found in {DATA_DIR}")
        print("Please check your DATA_DIR path!")
        return []
    
    available_records = []
    for dat_file in dat_files:
        # Extract record ID dari nama file (contoh: 04015.dat -> 04015)
        record_id = os.path.splitext(os.path.basename(dat_file))[0]
        
        # Pastikan file annotation (.atr) juga tersedia
        atr_file = os.path.join(DATA_DIR, f"{record_id}.atr")
        hea_file = os.path.join(DATA_DIR, f"{record_id}.hea")
        
        if os.path.exists(atr_file) and os.path.exists(hea_file):
            available_records.append(record_id)
        else:
            print(f"⚠️ Missing annotation or header file for {record_id}")
    
    print(f"✅ Found {len(available_records)} complete records")
    return sorted(available_records)

def analyze_single_record(record_id):
    """
    Analisis detail untuk satu record
    """
    try:
        # Load record dan annotation
        record_path = os.path.join(DATA_DIR, record_id)
        record = wfdb.rdrecord(record_path)
        ann = wfdb.rdann(record_path, "atr")
        
        # Basic info
        fs = record.fs
        duration_sec = record.sig_len / fs
        duration_hours = duration_sec / 3600
        
        # Annotation analysis
        if hasattr(ann, 'aux_note') and ann.aux_note is not None:
            labels = [label.strip() for label in ann.aux_note]
            unique_labels = set(labels)
            
            # Estimate windows (10-second windows with 50% overlap)
            estimated_windows = int((duration_sec / 10) * 2 - 1)  # Rough estimate with overlap
            
            # Classify AF vs Normal labels
            af_labels = {'(AFIB', 'AFIB'}
            normal_labels = {'(N', 'N', 'NSR'}
            
            af_annotations = [label for label in labels if label in af_labels]
            normal_annotations = [label for label in labels if label in normal_labels]
            other_annotations = [label for label in labels if label not in af_labels and label not in normal_labels]
            
            return {
                'record_id': record_id,
                'duration_hours': duration_hours,
                'duration_sec': duration_sec,
                'sampling_rate': fs,
                'total_annotations': len(labels),
                'unique_labels': unique_labels,
                'af_annotation_count': len(af_annotations),
                'normal_annotation_count': len(normal_annotations),
                'other_annotation_count': len(other_annotations),
                'estimated_windows': estimated_windows,
                'is_single_annotation': len(unique_labels) == 1,
                'predominant_rhythm': list(unique_labels)[0] if len(unique_labels) == 1 else 'Mixed',
                'all_labels': labels
            }
        else:
            print(f"⚠️ No annotations found for {record_id}")
            return None
            
    except Exception as e:
        print(f"❌ Error analyzing {record_id}: {e}")
        return None

def analyze_dataset_composition():
    """
    Analisis komposisi dataset lengkap
    """
    print("=" * 60)
    print("         DATASET COMPOSITION ANALYSIS")
    print("=" * 60)
    
    records = get_available_records()
    
    if not records:
        print("❌ No records found. Please check your DATA_DIR path.")
        return None, None
    
    # Analyze each record
    all_analyses = []
    single_annotation_records = []
    multi_annotation_records = []
    failed_records = []
    
    print(f"\n🔍 Analyzing {len(records)} records...")
    
    for i, record_id in enumerate(records, 1):
        print(f"  [{i:2d}/{len(records)}] Analyzing {record_id}...", end=" ")
        
        analysis = analyze_single_record(record_id)
        
        if analysis:
            all_analyses.append(analysis)
            
            if analysis['is_single_annotation']:
                single_annotation_records.append(analysis)
                print(f"✓ Single-annotation ({analysis['predominant_rhythm']})")
            else:
                multi_annotation_records.append(analysis)
                print(f"✓ Multi-annotation ({len(analysis['unique_labels'])} labels)")
        else:
            failed_records.append(record_id)
            print("❌ Failed")
    
    # Summary Statistics
    print(f"\n" + "=" * 60)
    print("                    SUMMARY")
    print("=" * 60)
    
    print(f"📊 Dataset Overview:")
    print(f"   Total records found: {len(records)}")
    print(f"   Successfully analyzed: {len(all_analyses)}")
    print(f"   Failed to analyze: {len(failed_records)}")
    
    if failed_records:
        print(f"   Failed records: {failed_records}")
    
    print(f"\n📈 Record Types:")
    print(f"   Single-annotation records: {len(single_annotation_records)}")
    print(f"   Multi-annotation records: {len(multi_annotation_records)}")
    
    # Detailed breakdown
    if single_annotation_records:
        print(f"\n🔍 Single-Annotation Records Detail:")
        
        af_single = [r for r in single_annotation_records if r['predominant_rhythm'] in ['(AFIB', 'AFIB']]
        normal_single = [r for r in single_annotation_records if r['predominant_rhythm'] in ['(N', 'N', 'NSR']]
        other_single = [r for r in single_annotation_records if r not in af_single and r not in normal_single]
        
        print(f"   AF-only records: {len(af_single)}")
        print(f"   Normal-only records: {len(normal_single)}")
        print(f"   Other-only records: {len(other_single)}")
        
        total_single_windows = sum(r['estimated_windows'] for r in single_annotation_records)
        af_single_windows = sum(r['estimated_windows'] for r in af_single)
        normal_single_windows = sum(r['estimated_windows'] for r in normal_single)
        
        print(f"   Total windows from single-annotation: {total_single_windows:,}")
        print(f"     AF windows: {af_single_windows:,}")
        print(f"     Normal windows: {normal_single_windows:,}")
        
        print(f"\n   Individual Record Details:")
        for record in sorted(single_annotation_records, key=lambda x: x['estimated_windows'], reverse=True):
            print(f"     {record['record_id']}: {record['predominant_rhythm']:8s} "
                  f"({record['estimated_windows']:4,} windows, {record['duration_hours']:5.1f}h)")
    
    if multi_annotation_records:
        print(f"\n🔍 Multi-Annotation Records Detail:")
        
        total_multi_windows = sum(r['estimated_windows'] for r in multi_annotation_records)
        print(f"   Total windows from multi-annotation: {total_multi_windows:,}")
        
        print(f"\n   Individual Record Details:")
        for record in sorted(multi_annotation_records, key=lambda x: x['estimated_windows'], reverse=True):
            labels_str = ', '.join(sorted(record['unique_labels']))
            print(f"     {record['record_id']}: {labels_str:20s} "
                  f"({record['estimated_windows']:4,} windows, {record['duration_hours']:5.1f}h)")
    
    # Impact Analysis
    if single_annotation_records and multi_annotation_records:
        single_windows = sum(r['estimated_windows'] for r in single_annotation_records)
        multi_windows = sum(r['estimated_windows'] for r in multi_annotation_records)
        total_windows = single_windows + multi_windows
        
        print(f"\n📊 Impact Analysis:")
        print(f"   Total estimated windows: {total_windows:,}")
        print(f"   From single-annotation: {single_windows:,} ({single_windows/total_windows:.1%})")
        print(f"   From multi-annotation: {multi_windows:,} ({multi_windows/total_windows:.1%})")
        
        # Check for dominant records
        if single_annotation_records:
            max_single_windows = max(r['estimated_windows'] for r in single_annotation_records)
            max_contribution = max_single_windows / total_windows
            
            if max_contribution > 0.3:
                print(f"   ⚠️  WARNING: One single-annotation record contributes {max_contribution:.1%} of total windows")
            
    return single_annotation_records, multi_annotation_records

def make_inclusion_decision(single_records, multi_records):
    """
    Automated decision berdasarkan dataset characteristics
    """
    print(f"\n" + "=" * 60)
    print("              INCLUSION DECISION ANALYSIS")
    print("=" * 60)
    
    if not single_records:
        print("❌ No single-annotation records found.")
        print("🎯 DECISION: N/A (no single-annotation records to include/exclude)")
        return "no_single_records"
    
    # Calculate metrics
    total_records = len(single_records) + len(multi_records)
    
    single_windows = sum(r['estimated_windows'] for r in single_records)
    multi_windows = sum(r['estimated_windows'] for r in multi_records) if multi_records else 0
    total_windows = single_windows + multi_windows
    
    single_ratio = single_windows / total_windows if total_windows > 0 else 0
    
    # Check class distribution
    af_single = [r for r in single_records if r['predominant_rhythm'] in ['(AFIB', 'AFIB']]
    normal_single = [r for r in single_records if r['predominant_rhythm'] in ['(N', 'N', 'NSR']]
    
    # Check for extreme records
    max_windows_per_record = max(r['estimated_windows'] for r in single_records)
    max_contribution = max_windows_per_record / total_windows if total_windows > 0 else 0
    
    # Decision logic
    risk_factors = []
    benefits = []
    warnings = []
    
    print(f"📊 Key Metrics:")
    print(f"   Total records: {total_records}")
    print(f"   Single-annotation contribution: {single_ratio:.1%} of total windows")
    print(f"   Max single record contribution: {max_contribution:.1%}")
    print(f"   AF-only records: {len(af_single)}")
    print(f"   Normal-only records: {len(normal_single)}")
    
    # Dataset size factor
    if total_records < 15:
        benefits.append(f"Small dataset ({total_records} records) - every record valuable")
    elif total_records > 25:
        risk_factors.append(f"Large dataset ({total_records} records) - single-annotation not essential")
    else:
        benefits.append(f"Medium dataset ({total_records} records) - additional data helpful")
    
    # Contribution factor  
    if single_ratio > 0.6:
        risk_factors.append(f"Very high single-annotation contribution ({single_ratio:.1%})")
    elif single_ratio > 0.4:
        warnings.append(f"High single-annotation contribution ({single_ratio:.1%}) - consider limits")
    elif single_ratio < 0.3:
        benefits.append(f"Reasonable single-annotation contribution ({single_ratio:.1%})")
    
    # Balance factor
    if len(af_single) > 0 and len(normal_single) > 0:
        benefits.append("Both AF and Normal single-annotation records available")
        af_single_windows = sum(r['estimated_windows'] for r in af_single)
        normal_single_windows = sum(r['estimated_windows'] for r in normal_single)
        af_ratio = af_single_windows / (af_single_windows + normal_single_windows)
        
        if 0.3 <= af_ratio <= 0.7:
            benefits.append(f"Balanced AF/Normal ratio in single-annotation ({af_ratio:.1%} AF)")
        else:
            warnings.append(f"Imbalanced AF/Normal in single-annotation ({af_ratio:.1%} AF)")
    else:
        risk_factors.append("Only one class in single-annotation records")
    
    # Dominance factor
    if max_contribution > 0.5:
        risk_factors.append(f"One record dominates dataset ({max_contribution:.1%})")
    elif max_contribution > 0.3:
        warnings.append(f"One record has high contribution ({max_contribution:.1%})")
    elif max_contribution < 0.2:
        benefits.append(f"No single record dominance ({max_contribution:.1%})")
    
    # Present analysis
    print(f"\n✅ Benefits of Including Single-Annotation Records:")
    if benefits:
        for i, benefit in enumerate(benefits, 1):
            print(f"   {i}. {benefit}")
    else:
        print("   None identified")
    
    print(f"\n⚠️  Potential Risks/Concerns:")
    if risk_factors:
        for i, risk in enumerate(risk_factors, 1):
            print(f"   {i}. {risk}")
    else:
        print("   No major risks identified")
    
    if warnings:
        print(f"\n🟡 Warnings (manageable with proper handling):")
        for i, warning in enumerate(warnings, 1):
            print(f"   {i}. {warning}")
    
    # Final decision
    risk_score = len(risk_factors)
    benefit_score = len(benefits)
    warning_score = len(warnings)
    
    print(f"\n🎯 DECISION ANALYSIS:")
    print(f"   Benefits: {benefit_score}")
    print(f"   Risks: {risk_score}")
    print(f"   Warnings: {warning_score}")
    
    if risk_score > benefit_score + 1:
        decision = "exclude"
        recommendation = "❌ EXCLUDE single-annotation records"
        reasoning = "Significant risks outweigh benefits"
    elif benefit_score > risk_score and warning_score <= 1:
        decision = "include"
        recommendation = "✅ INCLUDE single-annotation records"
        reasoning = "Benefits clearly outweigh risks"
    else:
        decision = "include_with_limits"
        recommendation = "🛡️ INCLUDE with sampling limits"
        reasoning = "Benefits present but need risk mitigation"
    
    print(f"\n" + "="*60)
    print(f"🎯 FINAL RECOMMENDATION: {recommendation}")
    print(f"📝 Reasoning: {reasoning}")
    
    if decision == "include_with_limits":
        print(f"\n💡 Suggested Limits:")
        suggested_limit = min(2000, int(total_windows * 0.15 / len(single_records)))
        print(f"   - Max windows per single-annotation record: {suggested_limit}")
        print(f"   - Max total contribution from single-annotation: 35%")
        print(f"   - Apply random sampling if records exceed limits")
    
    print("="*60)
    
    return decision

def save_analysis_results(single_records, multi_records, decision):
    """
    Save analysis results to CSV for reference
    """
    print(f"\n💾 Saving analysis results...")
    
    # Combine all records for CSV
    all_records_data = []
    
    for record in single_records + multi_records:
        all_records_data.append({
            'record_id': record['record_id'],
            'type': 'single_annotation' if record['is_single_annotation'] else 'multi_annotation',
            'predominant_rhythm': record['predominant_rhythm'],
            'unique_labels': ', '.join(sorted(record['unique_labels'])),
            'duration_hours': record['duration_hours'],
            'estimated_windows': record['estimated_windows'],
            'total_annotations': record['total_annotations']
        })
    
    df = pd.DataFrame(all_records_data)
    
    # Save to CSV
    output_file = os.path.join(os.path.dirname(DATA_DIR), 'dataset_analysis_results.csv')
    df.to_csv(output_file, index=False)
    
    print(f"✅ Results saved to: {output_file}")
    print(f"📋 Decision: {decision}")

def main():
    """
    Main analysis function
    """
    print("🩺 MIT-BIH AF Dataset Analysis Tool")
    print("📋 Analyzing dataset composition and single-annotation record impact\n")
    
    # Check if DATA_DIR exists
    if not os.path.exists(DATA_DIR):
        print(f"❌ Error: Directory '{DATA_DIR}' not found!")
        print("Please update DATA_DIR in the script to point to your dataset location.")
        return
    
    try:
        # Run analysis
        single_records, multi_records = analyze_dataset_composition()
        
        if single_records is not None:
            decision = make_inclusion_decision(single_records, multi_records)
            save_analysis_results(single_records, multi_records, decision)
            
            print(f"\n🎉 Analysis complete!")
            print(f"📄 Check the CSV file for detailed results.")
            print(f"💡 Use this analysis to inform your preprocessing decisions.")
        
    except Exception as e:
        print(f"❌ Error during analysis: {e}")
        print("Please check your dataset path and file permissions.")

if __name__ == "__main__":
    main()

🩺 MIT-BIH AF Dataset Analysis Tool
📋 Analyzing dataset composition and single-annotation record impact

         DATASET COMPOSITION ANALYSIS
Scanning for records in: D:\skripsi_teknis\dataset\mitbih-afdb
✅ Found 23 complete records

🔍 Analyzing 23 records...
  [ 1/23] Analyzing 04015... ✓ Multi-annotation (2 labels)
  [ 2/23] Analyzing 04043... ✓ Multi-annotation (3 labels)
  [ 3/23] Analyzing 04048... ✓ Multi-annotation (2 labels)
  [ 4/23] Analyzing 04126... ✓ Multi-annotation (2 labels)
  [ 5/23] Analyzing 04746... ✓ Multi-annotation (2 labels)
  [ 6/23] Analyzing 04908... ✓ Multi-annotation (3 labels)
  [ 7/23] Analyzing 04936... ✓ Multi-annotation (3 labels)
  [ 8/23] Analyzing 05091... ✓ Multi-annotation (2 labels)
  [ 9/23] Analyzing 05121... ✓ Multi-annotation (3 labels)
  [10/23] Analyzing 05261... ✓ Multi-annotation (2 labels)
  [11/23] Analyzing 06426... ✓ Multi-annotation (4 labels)
  [12/23] Analyzing 06453... ✓ Multi-annotation (2 labels)
  [13/23] Analyzing 06995... ✓ M